In [1]:
import numpy as np
import random
from neuralnetlib.layers import Input, Embedding, LSTM, Dense
from neuralnetlib.model import Model
from neuralnetlib.preprocessing import one_hot_encode
from neuralnetlib.callbacks import EarlyStopping

In [2]:
with open('dinos.txt', 'r', encoding='utf-8') as f:
    names = [line.strip() for line in f]

print(names[:5])  # on affiche les 5 premiers noms de la liste pour vois s'ils ont été correctement chargés

['Aachenosaurus', 'Aardonyx', 'Abdallahsaurus', 'Abelisaurus', 'Abrictosaurus']


In [3]:
lengths = [len(name) for name in names]
max_length = max(lengths)
print(f"Maximum length: {max_length}")

Maximum length: 26


In [4]:
# Constantes
PAD_TOKEN = ''  # Token de padding (index 0)
EOS_TOKEN = '$'  # Token de fin de séquence (index 1)
max_length = 10  # Longueur maximale des séquences

# Dictionnaires de mapping
char_to_index = {PAD_TOKEN: 0, EOS_TOKEN: 1}
index_to_char = {0: PAD_TOKEN, 1: EOS_TOKEN}

# Extraction des caractères uniques et tri
unique_chars = sorted(set(''.join(names)))

# Construction des mappings caractère <-> index en commençant à 2
for idx, char in enumerate(unique_chars, start=2):
    char_to_index[char] = idx
    index_to_char[idx] = char

vocab_size = len(char_to_index)
print(f"Vocab size: {vocab_size}")

Vocab size: 54


In [5]:
# Séquences pour le training
sequences = []
next_chars = []

# Création des séquences et des caractères suivants
for name in names:
    name = name.lower()
    name_chars = list(name) + [EOS_TOKEN]

    for i in range(len(name_chars) - 1):
        # Extraction de la séquence
        seq = name_chars[max(0, i - max_length + 1):i + 1]

        # Padding et conversion en indices
        padded_seq = [0] * (max_length - len(seq)) + [char_to_index[char] for char in seq]

        sequences.append(padded_seq)
        next_chars.append(char_to_index[name_chars[i + 1]])

# Conversion en arrays numpy
X = np.array(sequences)
y = one_hot_encode(np.array(next_chars), vocab_size)

print(f"Taille du vocabulaire: {vocab_size}")
print(f"Forme des données X: {X.shape}")
print(f"Forme des labels y: {y.shape}")

# Affichage d'un exemple bbpour vérification
print(f"\nExemple pour {names[0]}:")
print(f"Séquence d'entrée: {X[5]}")
print(f"Sortie attendue: {y[5]}")

# Visualisation des tokens pour le premier exemple
print("\nDécodage de la séquence d'exemple:")
print([index_to_char[idx] for idx in X[5]])
print(f"Prochain caractère: {index_to_char[next_chars[5]]}")

Taille du vocabulaire: 54
Forme des données X: (18374, 10)
Forme des labels y: (18374, 54)

Exemple pour Aachenosaurus:
Séquence d'entrée: [ 0  0  0  0 28 28 30 35 32 41]
Sortie attendue: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]

Décodage de la séquence d'exemple:
['', '', '', '', 'a', 'a', 'c', 'h', 'e', 'n']
Prochain caractère: o


In [6]:

# Création du modèle
embedding_dim = 32
lstm_units = 128

model = Model()
model.add(Input(max_length))
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(LSTM(units=lstm_units))
model.add(Dense(units=vocab_size, activation='softmax'))

model.compile(loss_function='categorical_crossentropy', optimizer='adam')
model.summary()

Model
-------------------------------------------------
Layer 1: Input(input_shape=(10,))
Layer 2: Embedding(input_dim=54, output_dim=32)
Layer 3: <neuralnetlib.layers.LSTM object at 0x000001F4948DF430>
Layer 4: <neuralnetlib.layers.Dense object at 0x000001F4948DF3D0>
Layer 5: Activation(Softmax)
-------------------------------------------------
Loss function: CategoricalCrossentropy
Optimizer: Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
-------------------------------------------------


In [7]:
# Création du callback EarlyStopping
early_stopping = EarlyStopping(
    monitor='loss',
    patience=5,
    restore_best_weights=True
)

# Entraînement du modèle
history = model.fit(
    X, y,
    epochs=100,
    batch_size=64,
    callbacks=[early_stopping],
    validation_data=(X, y),
)


[==============================] 100% Epoch 1/100 - loss: 2.6477 -  - 10.53s
[==============================] 100% Epoch 2/100 - loss: 2.0581 -  - 11.12s
[==============================] 100% Epoch 3/100 - loss: 1.8797 -  - 10.63s
[==============================] 100% Epoch 4/100 - loss: 1.7906 -  - 10.28s
[==============================] 100% Epoch 5/100 - loss: 1.7508 -  - 10.37s
[==============================] 100% Epoch 6/100 - loss: 1.7166 -  - 10.63s
[==============================] 100% Epoch 7/100 - loss: 1.6940 -  - 10.10s
[==============================] 100% Epoch 8/100 - loss: 1.6714 -  - 9.87s
[==============================] 100% Epoch 9/100 - loss: 1.6499 -  - 10.08s
[==============================] 100% Epoch 10/100 - loss: 1.6325 -  - 10.13s
[==============================] 100% Epoch 11/100 - loss: 1.6187 -  - 9.99s
[==============================] 100% Epoch 12/100 - loss: 1.5999 -  - 9.96s
[==============================] 100% Epoch 13/100 - loss: 1.5891 -  - 10.08

In [24]:
# Génération de nouveaux noms
def generate_name(model, min_length=5):
    current_sequence = [0] * max_length
    generated_name = ""

    while len(generated_name) < max_length:
        x = np.array([current_sequence])
        preds = model.predict(x)[0]

        # Sélection du prochain caractère avec random.choices
        next_char_idx = random.choices(range(vocab_size), weights=preds, k=1)[0]
        next_char = index_to_char[next_char_idx]

        # STOP si longueur minimale atteinte et EOS rencontré
        if len(generated_name) >= min_length and next_char == EOS_TOKEN:
            break

        # Ajout du caractère si ce n'est ni PAD ni EOS
        if next_char not in [PAD_TOKEN, EOS_TOKEN]:
            generated_name += next_char

        # Mise à jour de la séquence courante
        current_sequence = current_sequence[1:] + [next_char_idx]

    return generated_name.capitalize() if len(generated_name) >= min_length else None

# Génération de plusieurs noms
generated_names = []
number_of_names = 5
min_length = 5

while len(generated_names) < number_of_names:
    name = generate_name(model, min_length)
    if name is not None and name not in generated_names:
        generated_names.append(name)

# Affichage des résultats
print("\nNoms générés:")
for name in generated_names:
    print(f"{name} ({len(name)} caractères)")

# Vérification de l'originalité
print("\nTous les noms sont-ils originaux ?", all(name.lower() not in [n.lower() for n in names] for name in generated_names))

# Statistiques sur les longueurs
lengths = [len(name) for name in generated_names]
print(f"\nLongueur moyenne: {sum(lengths)/len(lengths):.1f} caractères")
print(f"Longueur minimale: {min(lengths)} caractères")
print(f"Longueur maximale: {max(lengths)} caractères")


Noms générés:
Ourocosaur (10 caractères)
Rsholisaur (10 caractères)
Cosonimus (9 caractères)
Euceratous (10 caractères)
Amarcerato (10 caractères)

Tous les noms sont-ils originaux ? True

Longueur moyenne: 9.8 caractères
Longueur minimale: 9 caractères
Longueur maximale: 10 caractères
